# Neural Art Bot Debate (notes and brainstorming)
The purpose of this notebook is to provide notes and a platform for brainstorming 
implementation details for a art installation which involves:

- Two neural art bot instances engaged in a debate on a variety of topics
- TTS to playback the arguments posed by each bot
- A display similar to CLI that shows the arguments for each bot in text format.
- (optional) A third bot that serves as a judge to the debate which allocates points to each side based on the merrits of their arugments

## Great, with these objectives in mind, lets start exploring our code options =)

Lets start off by using ChatGPT to drive the program. Later on we will build and use our own model to avoid the API costs associated with GPT.

While we might end up needing multiple instances of GPT, for now lets use multiple settings which are associated with the two bot instances

In [1]:
# Import the nessisary libraries
from tensorflow import keras
import tensorflow as tf
import openai
import json
import requests
import os
from random import shuffle, randint

# make sure we have the OpenAi API key stored as a 
# environment variable
assert os.environ.get('OPENAI_API_KEY') is not None, 'ERROR, your environment variable OPENAI_API_KEY is not set properly'

# load the API key into our instance of the openai library
# to make method calls simpler later on in the program
openai.api_key = os.getenv('OPENAI_API_KEY')

2023-04-15 13:52:59.666006: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["dlopen(/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so, 0x0006): symbol not found in flat namespace '__ZN3tsl6Status22MaybeAddSourceLocationENS_14SourceLocationE'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file 

In [2]:
# create an list to store the parameters 
# for our three personalities which take turns
#  serving as the two debaters and the judge
# TODO, consider generating the personality traits randomly using a master list of personality traits
personality_params = [
    {
        'name' : 'Jack',
        'gender' : "Male",
        'traits': ['articulate', 'liberal', 'passionate', 'logical'],
        'model': 'text-babbage-001',
        'temperature' : 1.0
    },
    {
        'name' : "Jill",
        'gender' : 'Female',
        'traits': ['reserved', 'conservative', 'naive', 'inquisitive'],
        'model': 'text-babbage-001',
        'temperature' : 0.8
    },
    {
        'name' : "Agustus",
        'gender' : 'Male',
        'traits': ['unbiased', 'neurotic', 'introverted', 'thoughtful'],
        'model': 'text-babbage-001',
        'temperature' : 1.2
    }
]

shuffle(personality_params)

# randomly shuffle the order of our personality_parms indices
# select one of the personality profiles at random for debater1
debater1_params = personality_params[0]
debater2_params = personality_params[1]
judge_params = personality_params[2]
# select one of the other profiles for debater 2

# Create a list of possible debate topics
topics = ["What is the cutest animal?", 
          "Who was the greatest president?",
          "What is the best way to cook a chicken dinner?",
          "What is the meaning of life?",
          "What is love?",
          "What is the most magical place on earth?"]

topic = topics[randint(0, len(topics)-1)]

print("-"*60)
print("Out first debater is {} : who has the traits of {}".format(debater1_params['name'], debater1_params['traits']))
print("Out second debater is {} : who has the traits of {}".format(debater2_params['name'], debater2_params['traits']))
print("Out Judge is {} : who has the traits of {}".format(judge_params['name'], judge_params['traits']))
print("The topic of our debate is: {}".format(topic))
print("-"*60)

------------------------------------------------------------
Out first debater is Jack : who has the traits of ['articulate', 'liberal', 'passionate', 'logical']
Out second debater is Jill : who has the traits of ['reserved', 'conservative', 'naive', 'inquisitive']
Out Judge is Agustus : who has the traits of ['unbiased', 'neurotic', 'introverted', 'thoughtful']
The topic of our debate is: What is the cutest animal?
------------------------------------------------------------


### Now that we have our two debaters, a judge, and a topic for debate, lets start the debate!

In [3]:
def sendDebateRequest(debater_params, topic, initial_call=False):
    max_tokens = 200
    # Create a dictionary to store our headers
    if initial_call is False:
        topic = 'Lets assume that you have the personality traits of {} and that this argument is incorrect,\n{}\n Please provide a counter argument in the first person that is fewer than {} words'.format(" ".join(debater_params['traits']), topic, max_tokens // 2)
    else:
        topic = "Assuming you are taking the role of someone with these personality traits: {}. Lets have a debate about the following topic: {}, can you please respond in under {} words in the first person?".format(" ".join(debater_params['traits']), topic, max_tokens // 2 )
    
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + os.environ.get('OPENAI_API_KEY')
    }
    # create the data dictionary for our API call
    data = {
        'model' : debater_params['model'],
        'temperature' : debater_params['temperature'],
        'n' : 1,
        'max_tokens' : max_tokens,
        'prompt' :  topic # TODO, what does the 'role' potion of this dict do?
         # 'stop' : ';'
    }
    # print("Our data dict is as follows: ", data)
    # print("Our header dict is as follows: {}".format(headers))
    # Pose the debate topic question to our first debater
    print("{} is generating a response to the prompt of: {}".format(debater_params['name'], topic))
    response = requests.post('https://api.openai.com/v1/completions', headers=headers, json=data).json()
    return response

In [4]:
responses = []
responses.append(sendDebateRequest(debater1_params, topic, initial_call=True))
print("The generated response is: ", responses[-1])
prompt = responses[-1]['choices'][0]['text']

Jack is generating a response to the prompt of: Assuming you are taking the role of someone with these personality traits: articulate liberal passionate logical. Lets have a debate about the following topic: What is the cutest animal?, can you please respond in under 100 words in the first person?
The generated response is:  {'id': 'cmpl-75hBJfjF4pMXPnERKmMhwKMpEsaHk', 'object': 'text_completion', 'created': 1681591985, 'model': 'text-babbage-001', 'choices': [{'text': '\n\nI cannot answer this question, because I do not know what the cutest animal is.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 47, 'completion_tokens': 20, 'total_tokens': 67}}


### Amazing, now that we have our first debater's statment, lets see what the counter argument is...

In [5]:
responses.append(sendDebateRequest(debater2_params, topic))
prompt = responses[-1]['choices'][0]['text']
print("The generated reply is: ", prompt)

Jill is generating a response to the prompt of: Lets assume that you have the personality traits of reserved conservative naive inquisitive and that this argument is incorrect,
What is the cutest animal?
 Please provide a counter argument in the first person that is fewer than 100 words
The generated reply is:  .

There is no cutest animal. Some animals are more cute than others and there is no universal answer.


## Amazing, now lets bring in the judge to determine who is more correct


In [6]:

def judgeArgument(debater1, debater2, judge, responses):
    debater_names = [debater1['name'], debater2['name']]
    max_tokens = 200
    # Create a dictionary to store our headers
    topic = 'I need you to judge which of two arguments are better according to the criteria of {}.\n Here is the first agument {}.\n Here is the second argument {}. Please respond with a single character consisting of either a "1" or "2".'.format(" ".join(judge['traits']), responses[-2], responses[-1])

    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + os.environ.get('OPENAI_API_KEY')
    }
    # create the data dictionary for our API call
    data = {
        'model' : judge['model'],
        'temperature' : judge['temperature'],
        'n' : 1,
        'max_tokens' : max_tokens,
        'prompt' :  topic # TODO, what does the 'role' potion of this dict do?
         # 'stop' : ';'
    }
    # print("Our data dict is as follows: ", data)
    # print("Our header dict is as follows: {}".format(headers))
    # Pose the debate topic question to our first debater
    print("Our Judge {} is generating a response to the prompt of: {}".format(judge['name'], topic))
    response = requests.post('https://api.openai.com/v1/completions', headers=headers, json=data).json()
    print("The judges response is: {}".format(response['choices'][0]['text']))
    return "The judge {} has determined that {} wins the debate.".format(judge['name'], debater_names[int(response['choices'][0]['text'])])

In [7]:
judgement = judgeArgument(debater1_params, debater2_params, judge_params, responses)
print(judgement)

Our Judge Agustus is generating a response to the prompt of: I need you to judge which of two arguments are better according to the criteria of unbiased neurotic introverted thoughtful.
 Here is the first agument {'id': 'cmpl-75hBJfjF4pMXPnERKmMhwKMpEsaHk', 'object': 'text_completion', 'created': 1681591985, 'model': 'text-babbage-001', 'choices': [{'text': '\n\nI cannot answer this question, because I do not know what the cutest animal is.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 47, 'completion_tokens': 20, 'total_tokens': 67}}.
 Here is the second argument {'id': 'cmpl-75hBKMMedHzBNQltLeZUJhZpOkGcG', 'object': 'text_completion', 'created': 1681591986, 'model': 'text-babbage-001', 'choices': [{'text': '.\n\nThere is no cutest animal. Some animals are more cute than others and there is no universal answer.', 'index': 0, 'logprobs': None, 'finish_reason': None}], 'usage': {'prompt_tokens': 47, 'completion_tokens': 24, 'total_tokens': 71}}. P

## Amazing, that completes our initial investigations!
Now lets create a function to run all the debate code together

In [8]:
def runDebate(debater1, debater2, judge, prompt):
    print("-"*60)
    print("Out first debater is {} : who has the traits of {}".format(debater1_params['name'], debater1_params['traits']))
    print("Out second debater is {} : who has the traits of {}".format(debater2_params['name'], debater2_params['traits']))
    print("Out Judge is {} : who has the traits of {}".format(judge_params['name'], judge_params['traits']))
    print("The topic of our debate is: {}".format(topic))
    print("-"*60)
    responses = []
    responses.append(sendDebateRequest(debater1_params, prompt, initial_call=True))
    print("The generated response is: ", responses[-1])
    prompt = responses[-1]['choices'][0]['text']
    responses.append(sendDebateRequest(debater2_params, prompt))
    prompt = responses[-1]['choices'][0]['text']
    print("The generated reply is: ", prompt)
    judgeArgument(debater1, debater2, judge, responses)
    

In [9]:
prompt = topics[randint(0, len(topics) - 1)]
runDebate(debater1_params, debater2_params, judge_params, prompt)

------------------------------------------------------------
Out first debater is Jack : who has the traits of ['articulate', 'liberal', 'passionate', 'logical']
Out second debater is Jill : who has the traits of ['reserved', 'conservative', 'naive', 'inquisitive']
Out Judge is Agustus : who has the traits of ['unbiased', 'neurotic', 'introverted', 'thoughtful']
The topic of our debate is: What is the cutest animal?
------------------------------------------------------------
Jack is generating a response to the prompt of: Assuming you are taking the role of someone with these personality traits: articulate liberal passionate logical. Lets have a debate about the following topic: What is love?, can you please respond in under 100 words in the first person?
The generated response is:  {'id': 'cmpl-75hBL2kuh4qvjHtNUcoaRpgHRMiPt', 'object': 'text_completion', 'created': 1681591987, 'model': 'text-babbage-001', 'choices': [{'text': "\n\nLove is a difficult concept to define. To many, it mi